In [23]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *

In [24]:
pp = sp.symbols('pp') # permit price
q1, q2 = sp.symbols('q1 q2') # quantity of firm1 and firm2
x1, x2 = sp.symbols('x1 x2') # emission of firm1 and firm2
cap = 0 # total emission cap

profit1 = (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**2 - pp*x1
profit2 = (10 - 0.1 * (q1 + q2)) * q2 - 100*(q2 - x2)**2 - pp*x2

# First order conditions
foc1 = sp.diff(profit1, q1)
foc2 = sp.diff(profit2, q2)
foc3 = sp.diff(profit1, x1)
foc4 = sp.diff(profit2, x2)

# Solve for q1, q2, x1, x2
solutions = sp.solve([foc1, foc2, foc3, foc4, x1 + x2 - cap], (q1, q2, x1, x2, pp))

print(solutions)

#Now again using a least squares approach and scipy optimization
print(foc1,"|", foc2,"|", foc3,"|", foc4)






{pp: 9.63159162051529, q1: 1.22802793161570, q2: 1.22802793161570, x1: -1.17986997351312, x2: 1.17986997351312}
-4.2*q1 - 0.1*q2 + 4*x1 + 10 | -0.1*q1 - 200.2*q2 + 200*x2 + 10 | -pp + 4*q1 - 4*x1 | -pp + 200*q2 - 200*x2


Παρακάτω λύνουμε το ίδιο βάζοντας ουσιαστικά κάθε μεταβλητή που μας ενδιαφέρει στο τετράγωνο. και ελαχιστοποιώντας το άθροισμά του. 

In [25]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1',lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2',lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp',lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn = foc1**2 + foc2**2 + foc3**2 + foc4**2 + (cap - x1 - x2)**2
example.setObjective(obj_fn, GRB.MINIMIZE)

example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5 columns and 6 nonzeros
Model fingerprint: 0xd94b7b5b
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 4e+03]
  QObjective range [2e+00, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolved: 0 rows, 3 columns, 0 nonzeros
Presolved model has 6 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual

Εδώ θα κάνουμε το ίδιο, αλλά αυτήν την φορά θα κάνουμε την δική μου ιδέα με τα constrains και το αθροισμα αλλού.

In [26]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1',lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2',lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp',lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn =   x1 + x2
example.setObjective(obj_fn, GRB.MAXIMIZE)
example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)
example.addConstr(foc1 == 0)
example.addConstr(foc2 == 0)
example.addConstr(foc3 == 0)
example.addConstr(foc4 == 0)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x4599d829
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model


AttributeError: Unable to retrieve attribute 'x'

In [22]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1',lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2',lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp',lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn =   (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**2 - pp*x1
example.setObjective(obj_fn, GRB.MAXIMIZE)
example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)
example.addConstr(foc1 == 0)
example.addConstr(foc2 == 0)
example.addConstr(foc3 == 0)
example.addConstr(foc4 == 0)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x2ff2b56e
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [1e+01, 1e+01]
  QObjective range [2e-01, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+01]

Continuous model is non-convex -- solving as a MIP

Presolve removed 3 rows and 1 columns
Presolve time: 0.00s
Presolved: 11 rows, 10 columns, 34 nonzeros
Presolved model has 2 quadratic constraint(s)
Presolved model has 3 bilinear constraint(s)
Variable types: 10 continuous, 0 integer (0 binary)

Root relaxation: infeasible, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds     

AttributeError: Unable to retrieve attribute 'x'